# Setup

In [23]:
import pandas as pd
from sortedcontainers import SortedSet

In [24]:
doc1 = 'This pasta is very tasty and affordable.'
doc2 = 'This pasta is not tasty and is affordable.'
doc3 = 'This pasta is delicious and cheap.'
doc4 = 'Pasta is tasty and pasta tastes good.'

## Tokenization

In [25]:
# This line defines a regular expression pattern tokenizer_re. The pattern [^a-zA-Z0-9] is used to match any character that is not a lowercase letter (a-z), an uppercase letter (A-Z), or a digit (0-9). The r before the string indicates a raw string in Python, which tells Python to interpret the backslashes in the string as literal characters and not as escape characters.
tokenizer_re = r"[^a-zA-Z0-9]"

In [26]:
# Split the sentence to a list of the words contained in the sentence (split by spaces between the words)

# This line imports Python's built-in re module, which provides support for regular expressions.
import re

def tokenize(doc: str) -> list():
    # שלב 1: נורמליזציה לאותיות קטנות
    lowercased = doc.lower()

    # שלב 2: החלפת תווים לא רצויים ברווחים
    cleaned = re.sub(tokenizer_re, " ", lowercased)

    # שלב 3: פיצול למילים (tokens)
    tokens = cleaned.split()

    # החזרת הרשימה הסופית של המילים
    return tokens

In [27]:
# For each document greate the tokenized representation of it
l_doc1 = tokenize(doc1)
l_doc2 = tokenize(doc2)
l_doc3 = tokenize(doc3)
l_doc4 = tokenize(doc4)

In [28]:
# Print the tokenization of the first document
l_doc1

['this', 'pasta', 'is', 'very', 'tasty', 'and', 'affordable']

# BOW

## Feature Extraction

In [29]:
# --- Create a Set of unique words per sentence ---
# SortedSet is a data structure that maintains unique elements in sorted order.
wordset = SortedSet()

# Each of these lines adds the words from a list (like l_doc1, l_doc2, etc.) to wordset. Since wordset is a set, it will only keep unique words and discard any duplicates.
wordset.update(l_doc1)
wordset.update(l_doc2)
wordset.update(l_doc3)
wordset.update(l_doc4)

SortedSet(['affordable', 'and', 'cheap', 'delicious', 'good', 'is', 'not', 'pasta', 'tastes', 'tasty', 'this', 'very'])

In [30]:
# --- Bag of Words Calculation ---
# This function takes two arguments: wordset, a set of unique words, and l_doc, a list of words
def calculate_bow(wordset, l_doc):
    bow_dict = dict.fromkeys(wordset, 0) # creates a dictionary
    for word in wordset: # iterates through each word
        bow_dict[word] = l_doc.count(word) # For each word, this line counts its occurrences in l_doc and updates the corresponding entry in tf_diz
    return bow_dict # The function returns the tf_diz dictionary, which now maps each word to its frequency in l_doc

In [31]:
import unittest

class TestCalculateBOW(unittest.TestCase):
    def test_normal_case(self):
        wordset = {"cat", "dog", "fish"}
        l_doc = ["cat", "dog", "dog", "fish", "cat", "cat"]
        expected = {"cat": 3, "dog": 2, "fish": 1}
        result = calculate_bow(wordset, l_doc)
        self.assertEqual(result, expected)

    def test_word_not_in_set(self):
        wordset = {"cat", "dog"}
        l_doc = ["cat", "dog", "mouse"]
        expected = {"cat": 1, "dog": 1}
        result = calculate_bow(wordset, l_doc)
        self.assertEqual(result, expected)

    def test_empty_wordset(self):
        wordset = set()
        l_doc = ["cat", "dog", "cat"]
        expected = {}
        result = calculate_bow(wordset, l_doc)
        self.assertEqual(result, expected)

    def test_empty_l_doc(self):
        wordset = {"cat", "dog", "fish"}
        l_doc = []
        expected = {"cat": 0, "dog": 0, "fish": 0}
        result = calculate_bow(wordset, l_doc)
        self.assertEqual(result, expected)

    def test_no_overlap(self):
        wordset = {"cat", "dog", "fish"}
        l_doc = ["bird", "elephant", "tiger"]
        expected = {"cat": 0, "dog": 0, "fish": 0}
        result = calculate_bow(wordset, l_doc)
        self.assertEqual(result, expected)

    def test_case_sensitivity(self):
        wordset = {"cat", "dog"}
        l_doc = ["Cat", "dog", "DOG", "cat"]
        expected = {"cat": 1, "dog": 1}  # 'Cat' and 'DOG' differ in case
        result = calculate_bow(wordset, l_doc)
        self.assertEqual(result, expected)

    def test_duplicate_words_in_wordset(self):
        wordset = {"cat", "dog", "cat"}  # Sets remove duplicate 'cat'
        l_doc = ["cat", "dog", "dog"]
        expected = {"cat": 1, "dog": 2}
        result = calculate_bow(wordset, l_doc)
        self.assertEqual(result, expected)

In [32]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestCalculateBOW)
runner = unittest.TextTestRunner(verbosity=2)
runner.run(suite)

test_case_sensitivity (__main__.TestCalculateBOW.test_case_sensitivity) ... ok
test_duplicate_words_in_wordset (__main__.TestCalculateBOW.test_duplicate_words_in_wordset) ... ok
test_empty_l_doc (__main__.TestCalculateBOW.test_empty_l_doc) ... ok
test_empty_wordset (__main__.TestCalculateBOW.test_empty_wordset) ... ok
test_no_overlap (__main__.TestCalculateBOW.test_no_overlap) ... ok
test_normal_case (__main__.TestCalculateBOW.test_normal_case) ... ok
test_word_not_in_set (__main__.TestCalculateBOW.test_word_not_in_set) ... ok

----------------------------------------------------------------------
Ran 7 tests in 0.004s

OK


<unittest.runner.TextTestResult run=7 errors=0 failures=0>

In [33]:
# --- Calculating Bag of Words for Documents ---
# Each call generates a dictionary (e.g., bow1, bow2) where keys are words from wordset and values are their frequencies in the respective document.
bow1 = calculate_bow(wordset, l_doc1)
bow2 = calculate_bow(wordset, l_doc2)
bow3 = calculate_bow(wordset, l_doc3)
bow4 = calculate_bow(wordset, l_doc4)

# BOW Calculation (pandas)

In [34]:
# --- Creating a DataFrame from BoW Data ---
df_bow = pd.DataFrame([bow1, bow2, bow3, bow4])
df_bow.head()

,affordable,and,cheap,delicious,good,is,not,pasta,tastes,tasty,this,very
0,1,1,0,0,0,1,0,1,0,1,1,1
1,1,1,0,0,0,2,1,1,0,1,1,0
2,0,1,1,1,0,1,0,1,0,0,1,0
3,0,1,0,0,1,1,0,2,1,1,0,0


# BOW Calculation (sklearn)

In [35]:
# --- Converting wordset to CountVectorizer datatype ---
# CountVectorizer is a popular tool in text analysis used to convert a collection of text documents into a matrix of token counts.
from sklearn.feature_extraction.text import CountVectorizer

# Converting wordset to CountVectorizer datatype
vectorizer = CountVectorizer(vocabulary=wordset)
print(vectorizer.get_feature_names_out())

['affordable' 'and' 'cheap' 'delicious' 'good' 'is' 'not' 'pasta' 'tastes'
 'tasty' 'this' 'very']


In [36]:
# fit_transform is a method that performs two functions: fit and transform.
# 1. Fit: identify all unique words in the documents
# 2. Transform: Each row of this matrix corresponds to one of the documents. The value in each cell of the matrix is the count of how many times the corresponding word appears in the respective document.
X = vectorizer.fit_transform([doc1,doc2,doc3,doc4])

# Creating a DataFrame from the Matrix (More user-friendly)
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
df_bow_sklearn.head()

,affordable,and,cheap,delicious,good,is,not,pasta,tastes,tasty,this,very
0,1,1,0,0,0,1,0,1,0,1,1,1
1,1,1,0,0,0,2,1,1,0,1,1,0
2,0,1,1,1,0,1,0,1,0,0,1,0
3,0,1,0,0,1,1,0,2,1,1,0,0


In [37]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform([doc1,doc2,doc3,doc4])
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
df_bow_sklearn.head()

,affordable,and,cheap,delicious,good,is,not,pasta,tastes,tasty,this,very
0,1,1,0,0,0,1,0,1,0,1,1,1
1,1,1,0,0,0,2,1,1,0,1,1,0
2,0,1,1,1,0,1,0,1,0,0,1,0
3,0,1,0,0,1,1,0,2,1,1,0,0


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform([doc1,doc2,doc3,doc4])
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
df_bow_sklearn.head()

,affordable,cheap,delicious,good,pasta,tastes,tasty
0,1,0,0,0,1,0,1
1,1,0,0,0,1,0,1
2,0,1,1,0,1,0,0
3,0,0,0,1,2,1,1


In [39]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words='english',ngram_range=(1,2))
X = vectorizer.fit_transform([doc1,doc2,doc3])
df_bow_sklearn = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())
df_bow_sklearn.head()

,affordable,cheap,delicious,delicious cheap,pasta,pasta delicious,pasta tasty,tasty,tasty affordable
0,1,0,0,0,1,0,1,1,1
1,1,0,0,0,1,0,1,1,1
2,0,1,1,1,1,1,0,0,0


# TFIDF

In [40]:
tfidf1 = "This movie is very scary and long"
tfidf2 = "This movie is not scary and is slow"
tfidf3 = "This movie is spooky and good"

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer(stop_words='english', ngram_range=(1,1))
X = vectorizer_tfidf.fit_transform([tfidf1, tfidf2, tfidf3])
df_tfidf_sklearn = pd.DataFrame(X.toarray(), columns=vectorizer_tfidf.get_feature_names_out())
df_tfidf_sklearn

,good,long,movie,scary,slow,spooky
0,0.000000,0.720333,0.425441,0.547832,0.000000,0.000000
1,0.000000,0.000000,0.425441,0.547832,0.720333,0.000000
2,0.652491,0.000000,0.385372,0.000000,0.000000,0.652491


In [45]:
wordset

SortedSet(['affordable', 'and', 'cheap', 'delicious', 'good', 'is', 'not', 'pasta', 'tastes', 'tasty', 'this', 'very'])

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer(vocabulary=wordset)
X = vectorizer_tfidf.fit_transform([tfidf1, tfidf2, tfidf3])
df_tfidf_sklearn = pd.DataFrame(X.toarray(), columns=vectorizer_tfidf.get_feature_names_out())
df_tfidf_sklearn

,affordable,and,cheap,delicious,good,is,not,pasta,tastes,tasty,this,very
0,0.0,0.412859,0.0,0.0,0.00000,0.412859,0.000000,0.0,0.0,0.0,0.412859,0.69903
1,0.0,0.335829,0.0,0.0,0.00000,0.671657,0.568607,0.0,0.0,0.0,0.335829,0.00000
2,0.0,0.412859,0.0,0.0,0.69903,0.412859,0.000000,0.0,0.0,0.0,0.412859,0.00000


In [43]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_tfidf = CountVectorizer()
X = vectorizer_tfidf.fit_transform([tfidf1, tfidf2, tfidf3])
df_tfidf_sklearn = pd.DataFrame(X.toarray(), columns=vectorizer_tfidf.get_feature_names_out())
df_tfidf_sklearn

,and,good,is,long,movie,not,scary,slow,spooky,this,very
0,1,0,1,1,1,0,1,0,0,1,1
1,1,0,2,0,1,1,1,1,0,1,0
2,1,1,1,0,1,0,0,0,1,1,0


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_tfidf = TfidfVectorizer()
X = vectorizer_tfidf.fit_transform([tfidf1, tfidf2, tfidf3])
df_tfidf_sklearn = pd.DataFrame(X.toarray(), columns=vectorizer_tfidf.get_feature_names_out())
df_tfidf_sklearn

,and,good,is,long,movie,not,scary,slow,spooky,this,very
0,0.296283,0.000000,0.296283,0.501651,0.296283,0.000000,0.381519,0.000000,0.000000,0.296283,0.501651
1,0.263600,0.000000,0.527200,0.000000,0.263600,0.446313,0.339433,0.446313,0.000000,0.263600,0.000000
2,0.320528,0.542701,0.320528,0.000000,0.320528,0.000000,0.000000,0.000000,0.542701,0.320528,0.000000
